In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [ ]:
import os
from langchain_ollama import ChatOllama

model_name="ministral-3:3b"
model_url=os.getenv('OLLAMA_HOST')


model = ChatOllama(
    model=model_name,
    api_base=model_url
)

model_name="granite4:1b"
model_summarize = ChatOllama(
    model=model_name,
    api_base=model_url
)

## Summarize messages

In [8]:
from langchain.agents import create_agent
from langgraph.checkpoint.memory import InMemorySaver
from langchain.agents.middleware import SummarizationMiddleware

agent = create_agent(
    model=model,
    checkpointer=InMemorySaver(),
    middleware=[
        SummarizationMiddleware(
            model=model_summarize,
            trigger=("tokens", 100),
            keep=("messages", 1)
        )
    ],
)

In [9]:
from langchain.messages import HumanMessage, AIMessage
from pprint import pprint

response = agent.invoke(
    {"messages": [
        HumanMessage(content="What is the capital of the moon?"),
        AIMessage(content="The capital of the moon is Lunapolis."),
        HumanMessage(content="What is the weather in Lunapolis?"),
        AIMessage(content="Skies are clear, with a high of 120C and a low of -100C."),
        HumanMessage(content="How many cheese miners live in Lunapolis?"),
        AIMessage(content="There are 100,000 cheese miners living in Lunapolis."),
        HumanMessage(content="Do you think the cheese miners' union will strike?"),
        AIMessage(content="Yes, because they are unhappy with the new president."),
        HumanMessage(content="If you were Lunapolis' new president how would you respond to the cheese miners' union?"),
        ]},
    {"configurable": {"thread_id": "1"}}
)

pprint(response)

{'messages': [HumanMessage(content="Here is a summary of the conversation to date:\n\nThere is a capital of the moon called Lunapolis. The weather in Lunapolis has clear skies with a high of 120C and a low of -100C. There are 100,000 cheese miners living in Lunapolis. Yes, the cheese miners' union will likely strike because they are unhappy with the new president.", additional_kwargs={}, response_metadata={}, id='57b2e005-25bf-475f-a97b-8aeea16ff700'),
              HumanMessage(content="If you were Lunapolis' new president how would you respond to the cheese miners' union?", additional_kwargs={}, response_metadata={}, id='aaae0d7a-9e40-44c6-82a1-4dff5daa2c16'),
              AIMessage(content='Ah, Lunapolis—the lunar metropolis where cheese miners brave extreme temperatures (120°C by day, -100°C by night!) to supply the galaxy’s most *aged* dairy products. As your new president, I’d approach the cheese miners’ strike with a mix of **diplomacy, policy creativity, and lunar pragmatism**

In [10]:
print(response["messages"][0].content)

Here is a summary of the conversation to date:

There is a capital of the moon called Lunapolis. The weather in Lunapolis has clear skies with a high of 120C and a low of -100C. There are 100,000 cheese miners living in Lunapolis. Yes, the cheese miners' union will likely strike because they are unhappy with the new president.


## Trim/delete messages

In [11]:
from typing import Any
from langchain.agents import AgentState
from langchain.messages import RemoveMessage
from langgraph.runtime import Runtime
from langchain.agents.middleware import before_agent
from langchain.messages import ToolMessage

@before_agent
def trim_messages(state: AgentState, runtime: Runtime) -> dict[str, Any] | None:
    """Remove all the tool messages from the state"""
    messages = state["messages"]

    tool_messages = [m for m in messages if isinstance(m, ToolMessage)]
    
    return {"messages": [RemoveMessage(id=m.id) for m in tool_messages]}

In [12]:
model_name="ministral-3:3b"
model = ChatOllama(
    model=model_name,
    api_base=model_url
)

In [13]:
agent = create_agent(
    model=model,
    checkpointer=InMemorySaver(),
    middleware=[trim_messages],
)

In [14]:
response = agent.invoke(
    {"messages": [
        HumanMessage(content="My device won't turn on. What should I do?"),
        ToolMessage(content="blorp-x7 initiating diagnostic ping…", tool_call_id="1"),
        AIMessage(content="Is the device plugged in and turned on?"),
        HumanMessage(content="Yes, it's plugged in and turned on."),
        ToolMessage(content="temp=42C voltage=2.9v … greeble complete.", tool_call_id="2"),
        AIMessage(content="Is the device showing any lights or indicators?"),
        HumanMessage(content="What's the temperature of the device?")
        ]},
    {"configurable": {"thread_id": "2"}}
)

# See how trim_messages removes ToolMessage from conversation
pprint(response)

{'messages': [HumanMessage(content="My device won't turn on. What should I do?", additional_kwargs={}, response_metadata={}, id='d126236e-6de4-4cce-ab30-4eec069d9745'),
              AIMessage(content='Is the device plugged in and turned on?', additional_kwargs={}, response_metadata={}, id='dfa2f096-e8c6-4771-92f0-cb57abb2f6c4'),
              HumanMessage(content="Yes, it's plugged in and turned on.", additional_kwargs={}, response_metadata={}, id='c8f45316-9320-446e-b658-defe1eadc1a3'),
              AIMessage(content='Is the device showing any lights or indicators?', additional_kwargs={}, response_metadata={}, id='0ad8dc9f-f056-4e04-b5e0-e8e53ca1a89d'),
              HumanMessage(content="What's the temperature of the device?", additional_kwargs={}, response_metadata={}, id='93d3f6a8-6137-4af5-8000-8fa9682affcf'),
              AIMessage(content='I can’t directly measure the temperature of your device or access its internal sensors. However, here are some steps you can take to diagn

In [15]:
print(response["messages"][-1].content)

I can’t directly measure the temperature of your device or access its internal sensors. However, here are some steps you can take to diagnose the issue:

### **1. Check for Overheating Symptoms**
- **Physical Signs**: If the device feels unusually hot to the touch (especially the back or bottom), it might be overheating.
- **Performance Issues**: If it’s running slowly, crashing, or behaving erratically, overheating could be a factor.

### **2. Common Causes of a Device Not Turning On Properly**
- **Power Supply Issues**: Try a different power adapter or outlet.
- **Faulty Battery**: If it’s a laptop/tablet, check if the battery is fully charged or replace it if damaged.
- **Hardware Failure**: A failing power button, motherboard issue, or short circuit can prevent it from turning on.
- **Software Glitch**: If it’s a smartphone, try a hard reset (hold power + volume down for a few seconds).

### **3. What to Do Next?**
- **Restart the Device**: Unplug it, wait 30 seconds, then plug it 